# VTuber Auto-Performance: Multi-Agent Edition

## 实验目标

**核心假设**: 真人主播的"人味"可以被结构化提取,迁移到AI主播上。

**新增特性**:
- Multi-Agent协作架构 (Planner → Performer → Director)
- Interruption Cost机制 (让AI学会"选择性忽略")
- Inner Monologue可视化 (暴露决策过程)

## 实验流程

```
Phase 1: 数据标注 -> Phase 2: 模式提炼 -> Phase 3: Multi-Agent执行 -> Phase 4: 结果分析
```

---

# 环境准备

In [1]:
!pip install openai anthropic langgraph -q

In [2]:
import os
import re
import json
import numpy as np
from typing import List, Tuple, Optional, Dict, TypedDict
from dataclasses import dataclass, field
from enum import Enum
from collections import Counter, defaultdict
from openai import OpenAI

# LangGraph imports
from langgraph.graph import StateGraph, END

print("✅ 依赖加载完成")

ModuleNotFoundError: No module named 'numpy'

---

# Schema定义

In [ ]:
# Schema定义
class AttentionFocus(str, Enum):
    SELF = "self"
    AUDIENCE = "audience"
    SPECIFIC = "specific"
    CONTENT = "content"
    META = "meta"

class SpeechAct(str, Enum):
    NARRATE = "narrate"
    OPINE = "opine"
    RESPOND = "respond"
    ELICIT = "elicit"
    PIVOT = "pivot"
    BACKCHANNEL = "backchannel"

class Trigger(str, Enum):
    SC = "sc"
    DANMAKU = "danmaku"
    SELF_INIT = "self"
    CONTENT = "content"
    PRIOR = "prior"

print("✅ Schema定义完成")
print(f"  AttentionFocus: {[e.value for e in AttentionFocus]}")
print(f"  SpeechAct: {[e.value for e in SpeechAct]}")
print(f"  Trigger: {[e.value for e in Trigger]}")

---

# 核心数据结构: Narrative Node

In [ ]:
@dataclass
class NarrativeNode:
    """叙事节点 - 包含打断代价的执行单元"""
    stage: str                    # Hook, Build-up, Climax, Resolution
    focus: str                    # self, audience, content
    goal: str                     # 核心任务描述
    interruption_cost: float      # 0.0-1.0: 被打断的代价
    speech_act_hint: List[str]    # 推荐的SpeechActs
    duration_hint: int            # 建议时长(秒)
    
    def get_current_cost(self, elapsed_ratio: float) -> float:
        """节点快结束时,cost自动降低"""
        if elapsed_ratio > 0.8:
            return self.interruption_cost * 0.5
        return self.interruption_cost

print("✅ NarrativeNode定义完成")

---

# LLM配置

In [ ]:
# LLM配置 - 改成你的API Key
LLM_CONFIG = {
    "api_key": os.getenv("ANTHROPIC_API_KEY", ""),
    "base_url": "https://api.anthropic.com/v1/",
    "model": "claude-sonnet-4-20250514",
}

def call_llm(prompt: str, max_tokens: int = 4096, temperature: float = 0.2) -> str:
    """统一的LLM调用接口"""
    client = OpenAI(
        api_key=LLM_CONFIG["api_key"],
        base_url=LLM_CONFIG["base_url"],
    )
    
    response = client.chat.completions.create(
        model=LLM_CONFIG["model"],
        max_tokens=max_tokens,
        temperature=temperature,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def robust_json_parse(text: str) -> dict:
    """鲁棒的JSON解析"""
    text = re.sub(r'```json\s*', '', text)
    text = re.sub(r'```\s*$', '', text)
    text = text.replace('"', '"').replace('"', '"')
    text = text.replace(''', "'").replace(''', "'")
    
    try:
        return json.loads(text)
    except json.JSONDecodeError as e:
        lines = text.split('\n')
        for i, line in enumerate(lines):
            if '"text":' in line:
                match = re.search(r'"text":\s*"(.*?)"(?=\s*,|\s*})', line)
                if match:
                    content = match.group(1)
                    fixed = content.replace('\\"', '###ESCAPED###')
                    fixed = fixed.replace('"', '\\"')
                    fixed = fixed.replace('###ESCAPED###', '\\"')
                    lines[i] = line.replace(content, fixed)
        return json.loads('\n'.join(lines))

print("✅ LLM配置完成")
print(f"  Model: {LLM_CONFIG['model']}")

---

# Phase 1: 数据处理

In [ ]:
def parse_timestamp(ts: str) -> float:
    parts = ts.strip().split(':')
    if len(parts) == 2:
        return int(parts[0]) * 60 + float(parts[1])
    elif len(parts) == 3:
        return int(parts[0]) * 3600 + int(parts[1]) * 60 + float(parts[2])
    return 0.0

def parse_raw_clips_markdown(md_content: str) -> List[dict]:
    """解析markdown格式的raw clips"""
    clips = []
    current_clip = None
    current_section = None
    transcript_lines = []
    notes_lines = []
    in_code_block = False
    
    for line in md_content.split('\n'):
        if line.strip() == '```':
            in_code_block = not in_code_block
            continue
        
        if line.startswith('## clip_'):
            if current_clip:
                current_clip['transcript_lines'] = transcript_lines
                current_clip['notes'] = '\n'.join(notes_lines)
                clips.append(current_clip)
            clip_id = line.replace('## ', '').strip()
            current_clip = {'id': clip_id, 'source': '', 'language': 'zh', 
                          'duration_sec': 120, 'title': ''}
            transcript_lines = []
            notes_lines = []
            current_section = None
            continue
        
        if not current_clip:
            continue
        
        if line.startswith('- **source**:'):
            current_clip['source'] = line.split(':', 1)[1].strip()
        elif line.startswith('- **language**:'):
            current_clip['language'] = line.split(':', 1)[1].strip()
        elif line.startswith('- **duration**:'):
            try:
                current_clip['duration_sec'] = int(line.split(':', 1)[1].strip())
            except:
                pass
        elif line.startswith('- **title**:'):
            current_clip['title'] = line.split(':', 1)[1].strip()
        elif line.startswith('### transcript'):
            current_section = 'transcript'
        elif line.startswith('### notes'):
            current_section = 'notes'
        elif current_section == 'transcript' and in_code_block and line.strip():
            match = re.match(r'^(\d{1,2}:\d{2})\s+(.+)$', line.strip())
            if match:
                transcript_lines.append({
                    'time': parse_timestamp(match.group(1)), 
                    'text': match.group(2)
                })
            elif transcript_lines:
                transcript_lines[-1]['text'] += ' ' + line.strip()
        elif current_section == 'notes' and line.strip() and not line.startswith('#'):
            notes_lines.append(line.strip())
    
    if current_clip:
        current_clip['transcript_lines'] = transcript_lines
        current_clip['notes'] = '\n'.join(notes_lines)
        clips.append(current_clip)
    
    return clips

print("✅ 数据解析器定义完成")

---

# Phase 2: Pattern Analysis

In [ ]:
class PatternAnalyzer:
    def __init__(self, clips: list):
        self.clips = clips
        self.all_segments = []
        for clip in clips:
            segs = clip.get('segments', []) if isinstance(clip, dict) else clip.segments
            self.all_segments.extend(segs)
    
    def compute_attention_transition_matrix(self) -> dict:
        transitions = defaultdict(lambda: defaultdict(int))
        for clip in self.clips:
            segs = clip.get('segments', []) if isinstance(clip, dict) else clip.segments
            for i in range(len(segs) - 1):
                f = segs[i].get('attention_focus', 'self') if isinstance(segs[i], dict) else segs[i].attention_focus.value
                t = segs[i+1].get('attention_focus', 'self') if isinstance(segs[i+1], dict) else segs[i+1].attention_focus.value
                transitions[f][t] += 1
        
        prob = {}
        for f, tos in transitions.items():
            total = sum(tos.values())
            prob[f] = {t: c/total for t, c in tos.items()}
        return prob
    
    def compute_trigger_speech_act_distribution(self) -> dict:
        dist = defaultdict(lambda: defaultdict(int))
        for seg in self.all_segments:
            tr = seg.get('trigger', 'self') if isinstance(seg, dict) else seg.trigger.value
            sa = seg.get('speech_act', 'narrate') if isinstance(seg, dict) else seg.speech_act.value
            dist[tr][sa] += 1
        
        prob = {}
        for tr, acts in dist.items():
            total = sum(acts.values())
            prob[tr] = {a: c/total for a, c in acts.items()}
        return prob
    
    def extract_catchphrases(self):
        cps = []
        for c in self.clips:
            cp = c.get('catchphrases', []) if isinstance(c, dict) else c.catchphrases
            if cp:
                cps.extend(cp)
        return Counter(cps).most_common(20)
    
    def infer_baseline_costs(self) -> dict:
        """从标注数据推断baseline cost值"""
        ignore_rates = defaultdict(lambda: {'ignored': 0, 'total': 0})
        
        for seg in self.all_segments:
            focus = seg.get('attention_focus', 'self')
            trigger = seg.get('trigger', 'self')
            
            if trigger == 'danmaku':
                ignore_rates[focus]['total'] += 1
                if seg.get('speech_act') != 'respond':
                    ignore_rates[focus]['ignored'] += 1
        
        baseline_costs = {}
        for focus, stats in ignore_rates.items():
            if stats['total'] > 0:
                baseline_costs[focus] = stats['ignored'] / stats['total']
            else:
                baseline_costs[focus] = 0.5
        
        return baseline_costs

print("✅ PatternAnalyzer定义完成")

---

# Phase 3: Multi-Agent System

In [ ]:
class PerformanceState(TypedDict):
    """Multi-Agent共享状态"""
    oc_setting: str
    topic: str
    skeleton: List[NarrativeNode]
    current_node_index: int
    danmaku_stream: List[dict]
    performance_log: List[dict]
    inner_monologues: List[str]
    elapsed_time: float
    baseline_costs: dict

print("✅ PerformanceState定义完成")

In [ ]:
class NarrativePlanner:
    """Agent 1: 叙事策划师"""
    
    def __init__(self, analyzer: PatternAnalyzer = None):
        self.analyzer = analyzer
        self.baseline_costs = analyzer.infer_baseline_costs() if analyzer else {}
        
        # 硬编码3个模板骨架
        self.templates = {
            "confessional": [
                NarrativeNode("Hook", "audience", "建立悬念/不寻常的开头", 0.3, ["elicit", "narrate"], 20),
                NarrativeNode("Build-up", "self", "铺垫细节/背景故事", 0.6, ["narrate"], 40),
                NarrativeNode("Climax", "self", "核心情绪爆发/秘密揭露", 0.95, ["narrate", "pivot"], 30),
                NarrativeNode("Resolution", "audience", "升华/寻求共鸣", 0.4, ["opine", "elicit"], 30)
            ],
            "debate": [
                NarrativeNode("Hook", "audience", "抛出争议话题", 0.2, ["elicit", "opine"], 15),
                NarrativeNode("Build-up", "content", "展示不同观点", 0.3, ["narrate", "opine"], 35),
                NarrativeNode("Climax", "audience", "与观众激烈互动", 0.2, ["respond", "opine"], 50),
                NarrativeNode("Resolution", "self", "总结立场", 0.5, ["opine"], 20)
            ],
            "insight": [
                NarrativeNode("Hook", "content", "提出有趣问题", 0.4, ["elicit"], 15),
                NarrativeNode("Build-up", "content", "理论/知识铺垫", 0.7, ["narrate"], 50),
                NarrativeNode("Climax", "self", "个人见解/经验", 0.6, ["opine", "narrate"], 35),
                NarrativeNode("Resolution", "audience", "实用建议", 0.3, ["opine", "elicit"], 20)
            ]
        }
    
    def plan(self, state: PerformanceState) -> PerformanceState:
        """生成带cost的skeleton"""
        oc = state['oc_setting']
        topic = state['topic']
        
        # 1. 选择模板
        template_key = "confessional"
        if any(w in topic.lower() for w in ['辩论', 'debate', '争议', '观点']):
            template_key = "debate"
        elif any(w in topic.lower() for w in ['分享', '技巧', '知识', 'insight']):
            template_key = "insight"
        
        skeleton = [NarrativeNode(
            n.stage, n.focus, n.goal, n.interruption_cost, 
            n.speech_act_hint, n.duration_hint
        ) for n in self.templates[template_key]]
        
        # 2. 个性化调整cost
        adjustment_prompt = f'''分析这个VTuber的性格特征:
{oc}

他们在讨论"{topic}"这个话题时,到了故事高潮阶段,会多大程度上忽略弹幕继续讲述?

请给出0.0-1.0的数值:
- 0.0 = 完全不介意被打断,很随性
- 0.5 = 会权衡,但可以被吸引走
- 1.0 = 绝对要讲完,谁都别想打断

只输出数字,不要解释。'''
        
        try:
            adjustment = float(call_llm(adjustment_prompt, max_tokens=10, temperature=0.1).strip())
            adjustment = max(0.0, min(1.0, adjustment))
            
            for node in skeleton:
                if node.stage == "Climax":
                    node.interruption_cost = adjustment
        except:
            pass
        
        # 3. 为每个节点生成具体目标
        for node in skeleton:
            goal_prompt = f'''VTuber设定: {oc}
话题: {topic}
阶段: {node.stage}

为这个阶段生成一个简短的叙事目标(20字内),例如:
- "用自嘲开场吸引注意"
- "讲述童年创伤经历"
- "引导观众分享经验"

只输出目标,不要解释:'''
            
            try:
                node.goal = call_llm(goal_prompt, max_tokens=50, temperature=0.7).strip()
            except:
                pass
        
        state['skeleton'] = skeleton
        state['baseline_costs'] = self.baseline_costs
        state['current_node_index'] = 0
        return state

print("✅ NarrativePlanner定义完成")

In [ ]:
class PerformerAgent:
    """Agent 2: 表演执行官"""
    
    def decide(self, state: PerformanceState) -> PerformanceState:
        """实时决策: 继续剧本 vs 回应弹幕"""
        if state['current_node_index'] >= len(state['skeleton']):
            return state
        
        node = state['skeleton'][state['current_node_index']]
        recent_danmaku = state['danmaku_stream'][-5:] if state['danmaku_stream'] else []
        
        # 1. 评估弹幕紧急度
        urgency = self._evaluate_urgency(recent_danmaku)
        
        # 2. 计算当前cost
        elapsed_in_node = state['elapsed_time'] - sum(
            state['skeleton'][i].duration_hint 
            for i in range(state['current_node_index'])
        )
        elapsed_ratio = elapsed_in_node / max(node.duration_hint, 1)
        current_cost = node.get_current_cost(elapsed_ratio)
        
        # 3. Agency决策公式
        decision_value = urgency - current_cost
        
        # 4. 生成台词
        if decision_value > 0.3:
            decision_type = "RESPOND"
            speech = self._generate_response(node, recent_danmaku, state)
        else:
            decision_type = "CONTINUE"
            speech = self._generate_script_speech(node, state)
        
        # 5. 记录
        state['performance_log'].append({
            "time": state['elapsed_time'],
            "node": node.stage,
            "decision": decision_type,
            "speech": speech,
            "urgency": urgency,
            "cost": current_cost,
            "decision_value": decision_value
        })
        
        # 6. 更新时间和节点
        state['elapsed_time'] += 10
        if state['elapsed_time'] >= sum(n.duration_hint for n in state['skeleton'][:state['current_node_index']+1]):
            state['current_node_index'] += 1
        
        return state
    
    def _evaluate_urgency(self, danmaku_list: List[dict]) -> float:
        """评估弹幕紧急度"""
        if not danmaku_list:
            return 0.0
        
        urgency = 0.0
        for dm in danmaku_list:
            dm_type = dm.get('type', 'normal')
            if dm_type == 'sc':  # Super Chat
                urgency = max(urgency, 0.9)
            elif dm_type == 'gift':
                urgency = max(urgency, 0.7)
            elif '?' in dm.get('text', '') or '？' in dm.get('text', ''):
                urgency = max(urgency, 0.5)
            else:
                urgency = max(urgency, 0.2)
        
        return urgency
    
    def _generate_response(self, node: NarrativeNode, danmaku: List[dict], state: PerformanceState) -> str:
        """生成回应弹幕的台词"""
        dm_texts = [dm.get('text', '') for dm in danmaku[-3:]]
        
        prompt = f'''你是一个VTuber,设定如下:
{state['oc_setting']}

当前正在讨论: {state['topic']}
当前阶段: {node.stage} - {node.goal}

观众发来的弹幕:
{chr(10).join(dm_texts)}

请用口语化的方式回应这些弹幕,保持角色设定。回应要简短有趣(30字以内)。
只输出台词,不要加引号或其他格式。'''
        
        try:
            return call_llm(prompt, max_tokens=100, temperature=0.8).strip()
        except:
            return "哎呀,等下再说这个~"
    
    def _generate_script_speech(self, node: NarrativeNode, state: PerformanceState) -> str:
        """生成剧本台词"""
        prompt = f'''你是一个VTuber,设定如下:
{state['oc_setting']}

当前话题: {state['topic']}
当前阶段: {node.stage}
这个阶段的目标: {node.goal}
推荐的说话方式: {', '.join(node.speech_act_hint)}

请生成一段符合当前阶段的台词(50-100字)。要口语化、有情绪起伏、符合角色设定。
只输出台词,不要加引号或其他格式。'''
        
        try:
            return call_llm(prompt, max_tokens=200, temperature=0.7).strip()
        except:
            return f"[{node.stage}阶段台词生成失败]"

print("✅ PerformerAgent定义完成")

In [ ]:
class DirectorAgent:
    """Agent 3: 导演 - 监督和调整表演"""
    
    def evaluate(self, state: PerformanceState) -> PerformanceState:
        """评估当前表演状态并记录内心独白"""
        if not state['performance_log']:
            return state
        
        recent_log = state['performance_log'][-3:]
        
        # 生成内心独白
        decisions = [log['decision'] for log in recent_log]
        avg_urgency = np.mean([log['urgency'] for log in recent_log])
        avg_cost = np.mean([log['cost'] for log in recent_log])
        
        monologue = f'''[导演视角] 
最近3次决策: {decisions}
平均弹幕紧急度: {avg_urgency:.2f}
平均打断代价: {avg_cost:.2f}
当前节点: {state['current_node_index'] + 1}/{len(state['skeleton'])}'''
        
        state['inner_monologues'].append(monologue)
        
        return state
    
    def should_continue(self, state: PerformanceState) -> bool:
        """判断是否继续表演"""
        return state['current_node_index'] < len(state['skeleton'])

print("✅ DirectorAgent定义完成")

---

# Phase 4: LangGraph工作流

In [ ]:
def create_performance_graph(planner: NarrativePlanner, performer: PerformerAgent, director: DirectorAgent):
    """创建Multi-Agent表演工作流"""
    
    # 定义节点函数
    def plan_node(state: PerformanceState) -> PerformanceState:
        return planner.plan(state)
    
    def perform_node(state: PerformanceState) -> PerformanceState:
        return performer.decide(state)
    
    def direct_node(state: PerformanceState) -> PerformanceState:
        return director.evaluate(state)
    
    def should_continue(state: PerformanceState) -> str:
        if director.should_continue(state):
            return "perform"
        return "end"
    
    # 构建图
    workflow = StateGraph(PerformanceState)
    
    workflow.add_node("plan", plan_node)
    workflow.add_node("perform", perform_node)
    workflow.add_node("direct", direct_node)
    
    workflow.set_entry_point("plan")
    workflow.add_edge("plan", "perform")
    workflow.add_edge("perform", "direct")
    workflow.add_conditional_edges(
        "direct",
        should_continue,
        {
            "perform": "perform",
            "end": END
        }
    )
    
    return workflow.compile()

print("✅ LangGraph工作流定义完成")

---

# 执行演示

In [ ]:
# 示例VTuber设定
DEMO_OC_SETTING = '''
名字: 小夜
性格: 元气满满但偶尔会突然emo,喜欢自嘲,说话很快
口癖: "诶嘿~", "这不是很正常吗", "等下等下"
特点: 容易被弹幕带跑,讲故事喜欢加戏剧化细节
'''

# 示例话题
DEMO_TOPIC = "分享一次社死经历"

# 模拟弹幕流
DEMO_DANMAKU = [
    {"time": 15, "text": "什么社死啊", "type": "normal"},
    {"time": 25, "text": "来了来了", "type": "normal"},
    {"time": 35, "text": "太真实了哈哈哈", "type": "normal"},
    {"time": 50, "text": "然后呢然后呢？", "type": "normal"},
    {"time": 65, "text": "主播加油", "type": "gift"},
    {"time": 80, "text": "我也有类似经历！", "type": "sc"},
    {"time": 95, "text": "笑死", "type": "normal"},
]

print("✅ 演示数据准备完成")
print(f"OC设定: {DEMO_OC_SETTING[:50]}...")
print(f"话题: {DEMO_TOPIC}")
print(f"弹幕数: {len(DEMO_DANMAKU)}")

In [ ]:
def run_performance_demo():
    """运行表演演示"""
    print("🎬 开始Multi-Agent表演演示\n")
    print("=" * 60)
    
    # 初始化Agents
    planner = NarrativePlanner(analyzer=None)
    performer = PerformerAgent()
    director = DirectorAgent()
    
    # 创建工作流
    graph = create_performance_graph(planner, performer, director)
    
    # 初始状态
    initial_state: PerformanceState = {
        "oc_setting": DEMO_OC_SETTING,
        "topic": DEMO_TOPIC,
        "skeleton": [],
        "current_node_index": 0,
        "danmaku_stream": [],
        "performance_log": [],
        "inner_monologues": [],
        "elapsed_time": 0.0,
        "baseline_costs": {}
    }
    
    # 运行工作流(带弹幕注入)
    danmaku_index = 0
    
    # 先运行planning
    state = planner.plan(initial_state)
    print("\n📋 叙事骨架生成完成:")
    for i, node in enumerate(state['skeleton']):
        print(f"  {i+1}. [{node.stage}] {node.goal} (cost={node.interruption_cost:.2f})")
    print()
    
    # 模拟表演循环
    max_iterations = 15
    for iteration in range(max_iterations):
        # 注入当前时间点的弹幕
        while danmaku_index < len(DEMO_DANMAKU) and DEMO_DANMAKU[danmaku_index]['time'] <= state['elapsed_time']:
            state['danmaku_stream'].append(DEMO_DANMAKU[danmaku_index])
            danmaku_index += 1
        
        # 执行一轮表演
        state = performer.decide(state)
        state = director.evaluate(state)
        
        # 输出最新的表演
        if state['performance_log']:
            latest = state['performance_log'][-1]
            print(f"⏱️ {latest['time']:.0f}s | [{latest['node']}] {latest['decision']}")
            print(f"   💬 {latest['speech'][:80]}..." if len(latest['speech']) > 80 else f"   💬 {latest['speech']}")
            print(f"   📊 urgency={latest['urgency']:.2f}, cost={latest['cost']:.2f}, decision_value={latest['decision_value']:.2f}")
            print()
        
        # 检查是否结束
        if not director.should_continue(state):
            print("\n🎬 表演结束!")
            break
    
    # 输出内心独白
    print("\n" + "=" * 60)
    print("🧠 导演内心独白:")
    for mono in state['inner_monologues'][-3:]:
        print(mono)
        print()
    
    return state

# 运行演示
# 注意: 需要配置有效的API Key才能运行
# final_state = run_performance_demo()

---

# 工具函数: 结果分析

In [ ]:
def analyze_performance(state: PerformanceState) -> dict:
    """分析表演结果"""
    if not state['performance_log']:
        return {"error": "No performance log"}
    
    logs = state['performance_log']
    
    # 统计决策分布
    decision_counts = Counter(log['decision'] for log in logs)
    
    # 计算平均值
    avg_urgency = np.mean([log['urgency'] for log in logs])
    avg_cost = np.mean([log['cost'] for log in logs])
    
    # 各阶段时长
    stage_times = defaultdict(list)
    for log in logs:
        stage_times[log['node']].append(log['time'])
    
    analysis = {
        "total_duration": state['elapsed_time'],
        "total_segments": len(logs),
        "decision_distribution": dict(decision_counts),
        "respond_rate": decision_counts.get('RESPOND', 0) / len(logs) if logs else 0,
        "avg_urgency": avg_urgency,
        "avg_cost": avg_cost,
        "stages_completed": len(stage_times),
        "danmaku_processed": len(state['danmaku_stream'])
    }
    
    return analysis

def print_analysis(analysis: dict):
    """打印分析结果"""
    print("📊 表演分析报告")
    print("=" * 40)
    print(f"总时长: {analysis['total_duration']:.0f}秒")
    print(f"总片段数: {analysis['total_segments']}")
    print(f"弹幕回应率: {analysis['respond_rate']:.1%}")
    print(f"平均弹幕紧急度: {analysis['avg_urgency']:.2f}")
    print(f"平均打断代价: {analysis['avg_cost']:.2f}")
    print(f"决策分布: {analysis['decision_distribution']}")

print("✅ 分析工具定义完成")

---

# 使用说明

## 快速开始

1. **配置API Key**: 修改 `LLM_CONFIG` 中的 `api_key`
2. **自定义OC设定**: 修改 `DEMO_OC_SETTING`
3. **设置话题**: 修改 `DEMO_TOPIC`
4. **运行演示**: 取消 `run_performance_demo()` 的注释

## 核心概念

### Interruption Cost (打断代价)
- 0.0-1.0 范围
- 高cost = 主播不想被打断,会忽略弹幕
- 低cost = 主播愿意互动

### 决策公式
```
decision_value = urgency - current_cost
if decision_value > 0.3: RESPOND (回应弹幕)
else: CONTINUE (继续剧本)
```

### 叙事骨架模板
- **confessional**: 告白型 (Hook→Build-up→Climax→Resolution)
- **debate**: 辩论型 (适合争议话题)
- **insight**: 知识分享型